In [1]:
import numpy as np
from vbydist import *

In [3]:
import numpy as np

# ------------------------------------------------------------------
# 1.  True distance-dependent velocity model  (concentric shells)
# ------------------------------------------------------------------
true_radial_model = [
    {'start': 0.0, 'end': 1.0,  'velocity': 4.0},   # 0–1 km
    {'start': 1.0, 'end': 2.0,  'velocity': 4.2},   # 1–2 km
    {'start': 2.0, 'end': 5.0,  'velocity': 4.5},   # 2–5 km
    {'start': 5.0, 'end': np.inf, 'velocity': 5.0}  # >5 km
]

# ------------------------------------------------------------------
# 2.  Ten stations: 5 on the surface + 5 down a borehole (same coords)
# ------------------------------------------------------------------
stations = [
    {'x': 0.0, 'y': 0.0, 'z': 0.0},
    {'x':10.0, 'y': 0.0, 'z': 0.0},
    {'x': 0.0, 'y':10.0, 'z': 0.0},
    {'x':10.0, 'y':10.0, 'z': 0.0},
    {'x': 5.0, 'y': 5.0, 'z': 0.0},   # borehole top
    {'x': 5.0, 'y': 5.0, 'z': 1.0},
    {'x': 5.0, 'y': 5.0, 'z': 3.0},
    {'x': 5.0, 'y': 5.0, 'z': 5.0},
    {'x': 5.0, 'y': 5.0, 'z': 7.0},
    {'x': 5.0, 'y': 5.0, 'z': 9.0},
]

# ------------------------------------------------------------------
# 3.  True event (unchanged)
# ------------------------------------------------------------------
true_event = {'x': 1.8, 'y': 2.4, 'z': 3.8, 't0': 0.5}

# ------------------------------------------------------------------
# 4.  Synthetic arrivals (straight-line travel, no ray tracing)
# ------------------------------------------------------------------

true_arrivals = []
print("\n--- Path diagnostics ---")
for sta in stations:
    t_calc, seg = compute_travel_time_radial(true_event, sta, true_radial_model)
    true_arrivals.append(t_calc + true_event['t0'])
    dist = sum(seg)        # same as np.linalg.norm(src-rec)
    vavg = dist / t_calc
    print(f"Sta ({sta['x']:4.1f},{sta['y']:4.1f},{sta['z']:4.1f})  "
          f"path = {dist:5.2f} km   t = {t_calc:6.3f} s   v̄ = {vavg:4.2f} km/s")

# ------------------------------------------------------------------
# 5.  Add Gaussian noise  (σ = 0.01 s)
# ------------------------------------------------------------------
np.random.seed(42)
observed_arrivals = [t + np.random.normal(0, 0.01) for t in true_arrivals]

# ------------------------------------------------------------------
# 6.  Initial guess (slightly off)
# ------------------------------------------------------------------
initial_event = {'x': 3.0, 'y': 3.0, 'z': 3.5, 't0': 0.4}

# ------------------------------------------------------------------
# 7.  Invert for location in the fixed radial model
# ------------------------------------------------------------------

estimated_event, rms_hist = invert_event_radial(
    event=initial_event,
    stations=stations,
    picks=observed_arrivals,
    velocity_model=true_radial_model,
    max_iter=500,
    tol=1e-4
)

# ------------------------------------------------------------------
# 8.  Report results
# ------------------------------------------------------------------
print("\n=== True event ===")
print(true_event)

print("\n=== Estimated event ===")
print(estimated_event)

print("\nRMS misfit history (s):", np.round(rms_hist, 5))



--- Path diagnostics ---
Sta ( 0.0, 0.0, 0.0)  path =  4.84 km   t =  1.120 s   v̄ = 4.32 km/s
Sta (10.0, 0.0, 0.0)  path =  9.35 km   t =  2.025 s   v̄ = 4.62 km/s
Sta ( 0.0,10.0, 0.0)  path =  8.69 km   t =  1.892 s   v̄ = 4.59 km/s
Sta (10.0,10.0, 0.0)  path = 11.81 km   t =  2.516 s   v̄ = 4.69 km/s
Sta ( 5.0, 5.0, 0.0)  path =  5.61 km   t =  1.276 s   v̄ = 4.39 km/s
Sta ( 5.0, 5.0, 1.0)  path =  4.98 km   t =  1.151 s   v̄ = 4.33 km/s
Sta ( 5.0, 5.0, 3.0)  path =  4.20 km   t =  0.977 s   v̄ = 4.30 km/s
Sta ( 5.0, 5.0, 5.0)  path =  4.29 km   t =  0.998 s   v̄ = 4.30 km/s
Sta ( 5.0, 5.0, 7.0)  path =  5.22 km   t =  1.199 s   v̄ = 4.35 km/s
Sta ( 5.0, 5.0, 9.0)  path =  6.64 km   t =  1.482 s   v̄ = 4.48 km/s

=== True event ===
{'x': 1.8, 'y': 2.4, 'z': 3.8, 't0': 0.5}

=== Estimated event ===
{'x': 1.8066718289059334, 'y': 2.372606461680736, 'z': 3.8048216870945746, 't0': 0.5027643845707578}

RMS misfit history (s): [0.29872 0.28741 0.27616 0.26496 0.25384 0.24278 0.23182 0.22

In [34]:
# ------------------------------------------------------------------
# 1.  TRUE distance shells
# ------------------------------------------------------------------
true_model = [
    {'start': 0.0, 'end': 2.0,   'velocity': 4.00},   # shell-1 (fixed later)
    {'start': 2.0, 'end': 6.0,   'velocity': 4.30},   # shell-2
    {'start': 6.0, 'end': np.inf,'velocity': 4.80},   # shell-3
]

# ------------------------------------------------------------------
# 2.  Sensor geometry – 4 corner boreholes + 1 deep + 6 wide surface
# ------------------------------------------------------------------
stations = []

# corner boreholes (z = 0, 2, 6 km)
for (x, y) in [(0, 0), (10, 0), (0, 10), (10, 10)]:
    for z in [0.0, 2.0, 6.0]:
        stations.append({'x': x, 'y': y, 'z': z})

# one deep borehole for long slant paths
for z in [8.0, 12.0]:
    stations.append({'x': 10.0, 'y': 0.0, 'z': z})

# far-field surface hexagon (r ≈ 15 km)
for (x, y) in [(15, 0), (7.5, 13), (-7.5, 13),
               (-15, 0), (-7.5, -13), (7.5, -13)]:
    stations.append({'x': x, 'y': y, 'z': 0.0})

print(f"Total stations: {len(stations)}")

# ------------------------------------------------------------------
# 3.  TRUE event
# ------------------------------------------------------------------
true_event = {'x': 2.0, 'y': 2.0, 'z': 3.0, 't0': 0.5}

# ------------------------------------------------------------------
# 4.  Synthetic arrivals (noise σ = 0.01 s)
# ------------------------------------------------------------------
arrivals = []
for sta in stations:
    t, _ = compute_travel_time_radial(true_event, sta, true_model)
    arrivals.append(t + true_event['t0'])

np.random.seed(42)
observed = [t + np.random.normal(0, 0.0) for t in arrivals]

# ------------------------------------------------------------------
# 5.  Initial guesses
# ------------------------------------------------------------------
init_event  = {'x': 3.0, 'y': 3.0, 'z': 2.5, 't0': 0.4}
init_model  = [
    {'start': 0.0, 'end': 2.0,   'velocity': 3.80},   # keep fixed
    {'start': 2.0, 'end': 6.0,   'velocity': 4.60},
    {'start': 6.0, 'end': np.inf,'velocity': 5.10},
]

# ------------------------------------------------------------------
# 6.  Joint inversion – FIX shell-1 by huge damping
# ------------------------------------------------------------------
damp_slow = np.array([1e6, 1e-3, 1e-3])  # shell-1,-2,-3
# invert_event_and_velocity_radial expects a scalar, so we’ll hack
# in a lambda that multiplies each row – simplest way: scale J columns
# (quick trick: patch inside the call with a pre-computed vector)
# → easiest: temporarily edit invert_event_and_velocity_radial or wrap it.
# Below we wrap:

def joint_inv(event, stations, picks, model, **kw):
    kw = {**kw, 'damp_slow': 1e-3}           # base damping
    est_evt, est_mod, rms = invert_event_and_velocity_radial(
        event, stations, picks, model, **kw
    )
    return est_evt, est_mod, rms

est_event, est_model, rms_hist = joint_inv(
    init_event, stations, observed, init_model,
    max_iter=1000, tol=1e-5, damp_xyz=1e-3, damp_t0=1e-3,
    max_dxyz=0.05, max_dv_rel=0.05, typeB_every=3
)

# ------------------------------------------------------------------
# 7.  Results
# ------------------------------------------------------------------
print("\n=== TRUE event ===")
print(true_event)

print("\n=== ESTIMATED event ===")
print(est_event)

print("\n=== UPDATED velocity model ===")
for lay in est_model:
    print(lay)

print("\nRMS misfit history (s):", np.round(rms_hist, 6))

Total stations: 20

=== TRUE event ===
{'x': 2.0, 'y': 2.0, 'z': 3.0, 't0': 0.5}

=== ESTIMATED event ===
{'x': 2.000000033411799, 'y': 2.000000033810485, 'z': 3.0000000093175627, 't0': 0.4664671312140473}

=== UPDATED velocity model ===
{'start': 0.0, 'end': 2.0, 'velocity': 3.7485977043544207}
{'start': 2.0, 'end': 6.0, 'velocity': 4.2999998427200525}
{'start': 6.0, 'end': inf, 'velocity': 4.80000002884275}

RMS misfit history (s): [3.03942e-01 2.17296e-01 1.81709e-01 1.66776e-01 1.57261e-01 1.47257e-01
 1.37583e-01 1.27866e-01 1.18280e-01 1.08768e-01 9.93790e-02 8.96120e-02
 7.98660e-02 7.00550e-02 6.01960e-02 5.02840e-02 4.03200e-02 3.03010e-02
 2.02280e-02 1.01020e-02 6.70000e-05 0.00000e+00]
